In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import scipy.io as spio
import h5py

Import Data from Matlab

In [2]:
filepath = '../ReflectionModel/dataset_multipath_profiles_one_hot_2m.mat'
arrays = {}
f = h5py.File(filepath)
for k, v in f.items():
    arrays[k] = np.array(v)

features = np.transpose(arrays['features'],(3,1,0,2))
labels = np.transpose(arrays['labels_one_hot'])
print(labels.shape)
print(features.shape)

(100000, 80)
(100000, 31, 41, 4)


Preprocess Data to make it 0-mean, 1 variance

In [3]:
#feature_mean = np.mean(features,axis=0)
feature_mean = np.mean(features)
feature_std = np.std(features)
feature_mean_m = np.tile(feature_mean, [features.shape[0],features.shape[1],features.shape[2],features.shape[3]])
feature_std_m = np.tile(feature_std, [features.shape[0],features.shape[1],features.shape[2],features.shape[3]])
features_normal = np.divide(features - feature_mean_m, feature_std_m)
print(feature_mean.shape)
print(feature_mean_m.shape)


()
(100000, 31, 41, 4)


Divide data into train, test and validation

In [4]:
train_pts = 60000
valid_pts = 20000
test_pts = 20000
features_normal = features_normal.astype('float32')
labels_normal = labels.astype('float32')
print(type(features_normal[0,0,0,0]))
labels_normal = labels_normal.astype('float32')
train_dataset = (features_normal[0:train_pts ,:,:,:])
train_labels = labels_normal[0:train_pts,:]
valid_dataset = features_normal[train_pts:valid_pts+train_pts,:,:,:]
valid_labels = labels_normal[train_pts:valid_pts+train_pts,:]
test_dataset = features_normal[train_pts+valid_pts:train_pts+valid_pts+test_pts,:,:,:]
test_labels = labels_normal[train_pts+valid_pts:train_pts+valid_pts+test_pts,:]
print(train_dataset.shape)
print(train_dataset[0,0,:,:])
print(train_labels[0,:])

<type 'numpy.float32'>
(60000, 31, 41, 4)
[[-0.32100356 -0.16091396 -0.43195084  0.09734473]
 [-0.32100356 -0.140065   -0.43023834  0.09449158]
 [-0.32100356 -0.10880438 -0.42264587  0.09134683]
 [-0.32100356 -0.08808839 -0.40795335  0.08791703]
 [-0.32100356 -0.05740891 -0.38971373  0.08420902]
 [-0.32100356 -0.03734463 -0.36622947  0.08022989]
 [-0.32100356 -0.00805223 -0.33742759  0.07598703]
 [-0.32100356  0.02000604 -0.30328035  0.07148801]
 [-0.32100356  0.04652648 -0.27079904  0.06674068]
 [-0.32100356  0.06320399 -0.23480465  0.06175307]
 [-0.32100356  0.08649906 -0.19554727  0.05653339]
 [-0.32100356  0.10747883 -0.16204199  0.04543154]
 [-0.32100356  0.12587866 -0.12694889  0.03956659]
 [-0.32100356  0.14598393 -0.09059244  0.02725257]
 [-0.32100356  0.15747036 -0.05336293  0.02082136]
 [-0.32100356  0.16568391 -0.02513981  0.00745574]
 [-0.32100356  0.17049006  0.01248187 -0.00652014]
 [-0.32100356  0.17144047  0.04038305 -0.02103327]
 [-0.32100356  0.16800165  0.05869231 -0

In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [11]:
batch_size = 256
patch_size = 5
depth = 32
num_hidden = 64
keep_prob = 0.8
decay_rate = 0.5
decay_steps = 1500
image_height = train_dataset.shape[1]
image_width = train_dataset.shape[2]
num_channels = train_dataset.shape[3]
num_labels = train_labels.shape[1]
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_height, image_width, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer2b_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2b_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [int(np.ceil(image_height / 4.0) * np.ceil(image_width / 4.0) * depth), num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  #def model(data):
  #  conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
  #  hidden = tf.nn.relu(conv + layer1_biases)
  #  conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
  #  hidden = tf.nn.relu(conv + layer2_biases)
  #  shape = hidden.get_shape().as_list()
  #  reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
  #  hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
  #  return tf.matmul(hidden, layer4_weights) + layer4_biases
  def model_for_accuracy(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='SAME')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='SAME')    
    conv = tf.nn.conv2d(hidden, layer2b_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2b_biases)
    hidden = tf.nn.max_pool(hidden,[1,2,2,1],[1,1,1,1],padding='SAME')    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    hidden = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='SAME')
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    hidden = tf.nn.max_pool(hidden,[1,2,2,1],[1,2,2,1],padding='SAME')    
    conv = tf.nn.conv2d(hidden, layer2b_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2b_biases)
    hidden = tf.nn.max_pool(hidden,[1,2,2,1],[1,1,1,1],padding='SAME')    
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
#    hidden = tf.nn.dropout(hidden,keep_prob)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model_for_accuracy(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model_for_accuracy(tf_test_dataset))

Run the optimizer

In [12]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
#    print(batch_labels[0,:])
#    print('hi')
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))      
    if (step % 500 == 0):
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))  
      print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.683907
Minibatch accuracy: 1.6%
Validation accuracy: 1.6%
Test accuracy: 1.6%
Minibatch loss at step 50: 2.125506
Minibatch accuracy: 43.0%
Minibatch loss at step 100: 1.157101
Minibatch accuracy: 71.1%
Minibatch loss at step 150: 0.611654
Minibatch accuracy: 79.3%
Minibatch loss at step 200: 0.701491
Minibatch accuracy: 74.2%
Minibatch loss at step 250: 0.346890
Minibatch accuracy: 87.9%
Minibatch loss at step 300: 0.518942
Minibatch accuracy: 78.9%
Minibatch loss at step 350: 0.416414
Minibatch accuracy: 83.6%
Minibatch loss at step 400: 0.360167
Minibatch accuracy: 84.4%
Minibatch loss at step 450: 0.305334
Minibatch accuracy: 91.0%
Minibatch loss at step 500: 0.340331
Minibatch accuracy: 86.3%
Validation accuracy: 89.1%
Test accuracy: 88.9%
Minibatch loss at step 550: 0.381699
Minibatch accuracy: 86.3%
Minibatch loss at step 600: 0.280012
Minibatch accuracy: 87.9%
Minibatch loss at step 650: 0.320290
Minibatch accuracy: 86.3%
Minibatch loss 